In [23]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms

from SDT import SDT

In [24]:
def onehot_coding(target, device, output_dim):
    """Convert the class labels into one-hot encoded vectors."""
    target_onehot = torch.FloatTensor(target.size()[0], output_dim).to(device)
    target_onehot.data.zero_()
    target_onehot.scatter_(1, target.view(-1, 1), 1.0)
    return target_onehot

In [25]:
input_dim = 28 * 28    # the number of input dimensions
output_dim = 10        # the number of outputs (i.e., # classes on MNIST)
depth = 5              # tree depth
lamda = 1e-3           # coefficient of the regularization term
lr = 1e-3              # learning rate
weight_decaly = 5e-4   # weight decay
batch_size = 128       # batch size
epochs = 3            # the number of training epochs
log_interval = 100     # the number of batches to wait before printing logs
use_cuda = False       # whether to use GPU

In [26]:
tree = SDT(input_dim, output_dim, depth, lamda, use_cuda)

In [27]:
optimizer = torch.optim.Adam(tree.parameters(),
                                lr=lr,
                                weight_decay=weight_decaly)


In [28]:
for param in tree.parameters():
    print(param.size())

torch.Size([31, 785])
torch.Size([10, 32])


In [29]:
data_dir = "../Dataset/mnist"

transformer = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]
)
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST(data_dir,
                    train=True,
                    download=True,
                    transform=transformer),
    batch_size=batch_size,
    shuffle=True,
)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST(data_dir,
                    train=False,
                    transform=transformer),
    batch_size=batch_size,
    shuffle=True,
)

In [33]:
best_testing_acc = 0.0
testing_acc_list = []
training_loss_list = []
criterion = nn.CrossEntropyLoss()
device = torch.device("cuda" if use_cuda else "cpu")

In [34]:
epoch = 0
tree.train()

SDT(
  (inner_nodes): Sequential(
    (0): Linear(in_features=785, out_features=31, bias=False)
    (1): Sigmoid()
  )
  (leaf_nodes): Linear(in_features=32, out_features=10, bias=False)
)

In [35]:

for batch_idx, (data, target) in enumerate(train_loader):

    batch_size = data.size()[0]
    data, target = data.to(device), target.to(device)
    target_onehot = onehot_coding(target, device, output_dim)

    output, penalty = tree.forward(data, is_training_data=True)

    loss = criterion(output, target.view(-1))
    loss += penalty

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print training status
    if batch_idx % log_interval == 0:
        pred = output.data.max(1)[1]
        correct = pred.eq(target.view(-1).data).sum()

        msg = (
            "Epoch: {:02d} | Batch: {:03d} | Loss: {:.5f} |"
            " Correct: {:03d}/{:03d}"
        )
        print(msg.format(epoch, batch_idx, loss, correct, batch_size))
        training_loss_list.append(loss.cpu().data.numpy())

# Evaluating
tree.eval()
correct = 0.

for batch_idx, (data, target) in enumerate(test_loader):

    batch_size = data.size()[0]
    data, target = data.to(device), target.to(device)

    output = F.softmax(tree.forward(data), dim=1)

    pred = output.data.max(1)[1]
    correct += pred.eq(target.view(-1).data).sum()

accuracy = 100.0 * float(correct) / len(test_loader.dataset)

if accuracy > best_testing_acc:
    best_testing_acc = accuracy

msg = (
    "\nEpoch: {:02d} | Testing Accuracy: {}/{} ({:.3f}%) |"
    " Historical Best: {:.3f}%\n"
)
print(
    msg.format(
        epoch, correct,
        len(test_loader.dataset),
        accuracy,
        best_testing_acc
    )
)
testing_acc_list.append(accuracy)


Epoch: 00 | Batch: 000 | Loss: 2.31331 | Correct: 013/128
Epoch: 00 | Batch: 100 | Loss: 2.08869 | Correct: 101/128
Epoch: 00 | Batch: 200 | Loss: 1.87784 | Correct: 114/128
Epoch: 00 | Batch: 300 | Loss: 1.73780 | Correct: 114/128
Epoch: 00 | Batch: 400 | Loss: 1.56026 | Correct: 115/128

Epoch: 00 | Testing Accuracy: 8894.0/10000 (88.940%) | Historical Best: 88.940%



In [20]:
for epoch in range(epochs):

    # Training
    tree.train()
    for batch_idx, (data, target) in enumerate(train_loader):

        batch_size = data.size()[0]
        data, target = data.to(device), target.to(device)
        target_onehot = onehot_coding(target, device, output_dim)

        output, penalty = tree.forward(data, is_training_data=True)

        loss = criterion(output, target.view(-1))
        loss += penalty

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print training status
        if batch_idx % log_interval == 0:
            pred = output.data.max(1)[1]
            correct = pred.eq(target.view(-1).data).sum()

            msg = (
                "Epoch: {:02d} | Batch: {:03d} | Loss: {:.5f} |"
                " Correct: {:03d}/{:03d}"
            )
            print(msg.format(epoch, batch_idx, loss, correct, batch_size))
            training_loss_list.append(loss.cpu().data.numpy())

    # Evaluating
    tree.eval()
    correct = 0.

    for batch_idx, (data, target) in enumerate(test_loader):

        batch_size = data.size()[0]
        data, target = data.to(device), target.to(device)

        output = F.softmax(tree.forward(data), dim=1)

        pred = output.data.max(1)[1]
        correct += pred.eq(target.view(-1).data).sum()

    accuracy = 100.0 * float(correct) / len(test_loader.dataset)

    if accuracy > best_testing_acc:
        best_testing_acc = accuracy

    msg = (
        "\nEpoch: {:02d} | Testing Accuracy: {}/{} ({:.3f}%) |"
        " Historical Best: {:.3f}%\n"
    )
    print(
        msg.format(
            epoch, correct,
            len(test_loader.dataset),
            accuracy,
            best_testing_acc
        )
    )
    testing_acc_list.append(accuracy)


Epoch: 00 | Batch: 000 | Loss: 1.98577 | Correct: 097/128
Epoch: 00 | Batch: 100 | Loss: 1.80180 | Correct: 102/128
Epoch: 00 | Batch: 200 | Loss: 1.65400 | Correct: 109/128
Epoch: 00 | Batch: 300 | Loss: 1.57277 | Correct: 102/128
Epoch: 00 | Batch: 400 | Loss: 1.43275 | Correct: 098/128

Epoch: 00 | Testing Accuracy: 7918.0/10000 (79.180%) | Historical Best: 79.180%

Epoch: 01 | Batch: 000 | Loss: 1.39647 | Correct: 101/128
Epoch: 01 | Batch: 100 | Loss: 1.29915 | Correct: 101/128
Epoch: 01 | Batch: 200 | Loss: 1.15956 | Correct: 107/128
Epoch: 01 | Batch: 300 | Loss: 1.14269 | Correct: 106/128
Epoch: 01 | Batch: 400 | Loss: 1.14038 | Correct: 095/128

Epoch: 01 | Testing Accuracy: 8173.0/10000 (81.730%) | Historical Best: 81.730%

Epoch: 02 | Batch: 000 | Loss: 0.95613 | Correct: 111/128
Epoch: 02 | Batch: 100 | Loss: 0.95063 | Correct: 109/128
Epoch: 02 | Batch: 200 | Loss: 0.77769 | Correct: 110/128
Epoch: 02 | Batch: 300 | Loss: 0.79426 | Correct: 111/128
Epoch: 02 | Batch: 400 |